In [ ]:
import contextily as cx
import xyzservices.providers as xyz
import matplotlib.pyplot as plt
import geopandas as gpd
import pyproj
import rasterio as rio
import geopandas as gpd #pip install geopandas descartes
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [ ]:
def reproject_raster(in_path, out_path, dst_crs, bounds):

    # reproject raster to project crs
    with rio.open(in_path) as src:
        src_crs = src.crs
        print(f"Current projection: {src_crs}")
        transform, width, height = calculate_default_transform(src_crs, dst_crs, src.width, src.height, *bounds)
        kwargs = src.meta.copy()

        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height})

        with rio.open(out_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rio.band(src, i),
                    destination=rio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.bilinear)
            print(f"New projection: {dst.crs}")
    return out_path

In [ ]:
RASTER_DIR = "../raster_maps/"
areas = [
    'Centro',   
    'Oeste',
    'Sul1',
    'Sul2',
    'Norte1',
    'Norte2',
    'Leste1',
    'Leste2'
]

crs = pyproj.CRS({'proj': 'longlat', 'ellps': 'WGS84', 'no_defs': True})      


for area in areas:
    db = gpd.read_file(f"../new_maps/{area}_modified/{area}_modified.shp")
    db.to_crs({'init': 'epsg:3857'}, inplace=True)

    w, s, e, n = db.total_bounds
    img, ext = cx.bounds2raster(w, s, e, n, f"{RASTER_DIR}{area}.tiff", source=xyz.CartoDB.Positron)
    reproject_raster(f"{RASTER_DIR}{area}.tiff", f"{RASTER_DIR}{area}_modified.tiff", crs, db.total_bounds)
